In [1]:
import pandas as pd
income_train = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data',sep=',\s',header = None, 
                    na_values=['?'], engine ='python')
income_test = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test', skiprows=[0], 
                    sep=',\s',header = None, na_values=['?'], engine = 'python')

# set to display all columns in running result
pd.set_option('display.max_columns', 50)

# To return all running results in Jupyter Notebook
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

income_train.head()
income_test.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K.
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K.
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K.
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K.
4,18,NaN,103497,Some-college,10,Never-married,NaN,Own-child,White,Female,0,0,30,United-States,<=50K.


In [2]:
#data.shape
#data2.shape
frames = [income_train, income_test]
census = pd.concat(frames)
census.shape
census.head()

(48842, 15)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [3]:
census.columns = ['age', 'workclass', 'fnlwgt', 'education','education_num','marital-status','occupation',
                  'relationship','race','sex','capital_gain','capital_loss','hours_per_week','native_country','income' ]

In [4]:
census = census.dropna()
census.shape

(45222, 15)

In [5]:
census = census.drop_duplicates()
census.shape

(45194, 15)

In [6]:
# replace the dot with empty in income column
census['income'] = census['income'].str.replace(".", "")

census['income'].value_counts()

<=50K    33988
>50K     11206
Name: income, dtype: int64

In [7]:
# convert target variable to binary
income_mapping = {'<=50K': [0], '>50K':[1]}

census['income'] = census['income'].replace(income_mapping)

census['income'].value_counts()

0    33988
1    11206
Name: income, dtype: int64

In [8]:
# Native country mapping
countries_mapping = pd.read_csv('Native Country Mapping.csv')

# countries_mapping['native_country'].astype(str)
# countries_mapping.dtypes
#countries_mapping.set_index('native_country').to_dict()['native_country_new']

countries_mapping['native_country'] = countries_mapping['native_country'].str.strip()

census = pd.merge(census,countries_mapping,on='native_country',how='left')
#census.merge(countries_mapping,left_on='native_country',how='left')
#census['native_country_new'] = census['native_country'].replace(countries_mapping)

In [9]:
# Education mapping
education_mapping = pd.read_csv('Education mapping.csv')

education_mapping['education'] = education_mapping['education'].str.strip()

census = pd.merge(census,education_mapping,on='education',how='left')

census.head()

,age,workclass,fnlwgt,education,education_num,marital-status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income,native_country_new,education_new
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,0,US,Bachelors
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,0,US,Bachelors
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,0,US,HS-grad
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,0,US,HS-dropouts
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,0,South America,Bachelors


In [10]:
# Create a new column net gain
census['net_capital_gain'] = census['capital_gain'] - census['capital_loss']

census.head()

,age,workclass,fnlwgt,education,education_num,marital-status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income,native_country_new,education_new,net_capital_gain
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,0,US,Bachelors,2174
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,0,US,Bachelors,0
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,0,US,HS-grad,0
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,0,US,HS-dropouts,0
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,0,South America,Bachelors,0


In [12]:
# Convert to dummy variables

census = pd.get_dummies(data=census, columns=['workclass', 'education_new', 'marital-status', 'occupation', 'relationship', 'sex', 'native_country_new'])

census.head()

,age,fnlwgt,education,education_num,race,capital_gain,capital_loss,hours_per_week,native_country,income,net_capital_gain,workclass_Federal-gov,workclass_Local-gov,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,workclass_State-gov,workclass_Without-pay,education_new_Assoc-acdm,education_new_Assoc-voc,education_new_Bachelors,education_new_Doctorate,education_new_HS-dropouts,education_new_HS-grad,education_new_Masters,...,occupation_Craft-repair,occupation_Exec-managerial,occupation_Farming-fishing,occupation_Handlers-cleaners,occupation_Machine-op-inspct,occupation_Other-service,occupation_Priv-house-serv,occupation_Prof-specialty,occupation_Protective-serv,occupation_Sales,occupation_Tech-support,occupation_Transport-moving,relationship_Husband,relationship_Not-in-family,relationship_Other-relative,relationship_Own-child,relationship_Unmarried,relationship_Wife,sex_Female,sex_Male,native_country_new_Asia,native_country_new_Europe,native_country_new_North America,native_country_new_South America,native_country_new_US
0,39,77516,Bachelors,13,White,2174,0,40,United-States,0,2174,0,0,0,0,0,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1
1,50,83311,Bachelors,13,White,0,0,13,United-States,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1
2,38,215646,HS-grad,9,White,0,0,40,United-States,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1
3,53,234721,11th,7,Black,0,0,40,United-States,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1
4,28,338409,Bachelors,13,Black,0,0,40,Cuba,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0


In [15]:
census_clean = census.drop(axis=1, columns=['fnlwgt', 'education', 'capital_gain', 'capital_loss', 'native_country'])

census.head()

,age,fnlwgt,education,education_num,race,capital_gain,capital_loss,hours_per_week,native_country,income,net_capital_gain,workclass_Federal-gov,workclass_Local-gov,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,workclass_State-gov,workclass_Without-pay,education_new_Assoc-acdm,education_new_Assoc-voc,education_new_Bachelors,education_new_Doctorate,education_new_HS-dropouts,education_new_HS-grad,education_new_Masters,...,occupation_Craft-repair,occupation_Exec-managerial,occupation_Farming-fishing,occupation_Handlers-cleaners,occupation_Machine-op-inspct,occupation_Other-service,occupation_Priv-house-serv,occupation_Prof-specialty,occupation_Protective-serv,occupation_Sales,occupation_Tech-support,occupation_Transport-moving,relationship_Husband,relationship_Not-in-family,relationship_Other-relative,relationship_Own-child,relationship_Unmarried,relationship_Wife,sex_Female,sex_Male,native_country_new_Asia,native_country_new_Europe,native_country_new_North America,native_country_new_South America,native_country_new_US
0,39,77516,Bachelors,13,White,2174,0,40,United-States,0,2174,0,0,0,0,0,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1
1,50,83311,Bachelors,13,White,0,0,13,United-States,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1
2,38,215646,HS-grad,9,White,0,0,40,United-States,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1
3,53,234721,11th,7,Black,0,0,40,United-States,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1
4,28,338409,Bachelors,13,Black,0,0,40,Cuba,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0
